# Operaciones de cadenas vectorizadas

Una fortaleza de Python es su relativa facilidad para manejar y manipular datos de cadenas.
Pandas se basa en esto y proporciona un conjunto completo de *operaciones de cadenas vectorizadas* que son una parte importante del tipo de manipulación necesaria cuando se trabaja con (léase: limpieza) datos del mundo real.
En este capítulo, analizaremos algunas de las operaciones de cadenas de Pandas y luego veremos cómo usarlas para limpiar parcialmente un conjunto de datos muy desordenado de recetas recopiladas de Internet.

## Presentamos las operaciones de cadenas de Pandas

Vimos en capítulos anteriores cómo herramientas como NumPy y Pandas generalizan operaciones aritméticas para que podamos realizar fácil y rápidamente la misma operación en muchos elementos de una matriz. Por ejemplo:

In [1]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

Esta *vectorización* de operaciones simplifica la sintaxis de operar con matrices de datos: ya no tenemos que preocuparnos por el tamaño o la forma de la matriz, sino solo por qué operación queremos que se realice.
Para matrices de cadenas, NumPy no proporciona un acceso tan simple y, por lo tanto, no puede utilizar una sintaxis de bucle más detallada:

In [2]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

Quizás esto sea suficiente para trabajar con algunos datos, pero se interrumpirá si faltan valores, por lo que este enfoque requiere realizar comprobaciones adicionales:

In [3]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']
[s if s is None else s.capitalize() for s in data]

['Peter', 'Paul', None, 'Mary', 'Guido']

Este tipo de enfoque manual no sólo es detallado e inconveniente, sino que también puede ser propenso a errores.

Pandas incluye características para abordar tanto esta necesidad de operaciones de cadenas vectorizadas como la necesidad de manejar correctamente los datos faltantes a través del atributo `str` de los objetos `Series` e `Index` de Pandas que contienen cadenas.
Entonces, por ejemplo, si creamos una `Serie` de Pandas con estos datos, podemos llamar directamente al método `str.capitalize`, que tiene incorporado el manejo de valores faltantes:

In [4]:
import pandas as pd
names = pd.Series(data)
names.str.capitalize()

0    Peter
1     Paul
2     None
3     Mary
4    Guido
dtype: object

## Tablas de métodos de cadena de Pandas

Si tiene un buen conocimiento de la manipulación de cadenas en Python, la mayor parte de la sintaxis de cadenas de Pandas es lo suficientemente intuitiva como para que probablemente sea suficiente enumerar los métodos disponibles. Comenzaremos con eso aquí, antes de profundizar en algunas de las sutilezas.
Los ejemplos de esta sección utilizan el siguiente objeto "Serie":

In [5]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])

### Métodos similares a los métodos de cadena de Python

Casi todos los métodos de cadena integrados de Python se reflejan en un método de cadena vectorizado de Pandas. Aquí hay una lista de métodos `str` de Pandas que reflejan los métodos de cadena de Python:

|           |                |                |                |
|-----------|----------------|----------------|--- -------------|
|`len()` | `inferior()` | `traducir()` | `islower()` | 
|`ljust()` | `superior()` | `comienza con()` | `issuperior()` | 
|`rjust()` | `buscar()` | `termina con()` | `isnumeric()` | 
|`centro()` | `rfind()` | `isalnum()` | `isdecimal()` | 
|`zfill()` | `índice()` | `isalfa()` | `dividir()` | 
|`tira()` | `índice()` | `isdigit()` | `rsplit()` | 
|`rstrip()` | `capitalizar()` | `isespacio()` | `partición()` | 
|`lstrip()` | `cambio()` | `istítulo()` | `rpartición()` |

Observe que estos tienen varios valores de retorno. Algunos, como "inferior", devuelven una serie de cadenas:

In [6]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

Pero algunos otros devuelven números:

In [7]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

O valores booleanos:

In [8]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

Otros devuelven listas u otros valores compuestos para cada elemento:

In [9]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

Veremos más manipulaciones de este tipo de objeto de serie de listas a medida que continuemos nuestra discusión.

### Métodos que utilizan expresiones regulares

Además, existen varios métodos que aceptan expresiones regulares (regexps) para examinar el contenido de cada elemento de cadena y siguen algunas de las convenciones API del módulo `re` integrado de Python:

| Método | Descripción |
|-----------|-------------|
| `partido` | Llama a `re.match` en cada elemento y devuelve un valor booleano. |
| `extraer` | Llama a `re.match` en cada elemento y devuelve grupos coincidentes como cadenas.|
| `encontrar` | Llama a `re.findall` en cada elemento |
| `reemplazar` | Reemplaza las apariciones del patrón con alguna otra cadena|
| `contiene`| Llama a `re.search` en cada elemento y devuelve un valor booleano |
| `contar` | Cuenta las apariciones del patrón|
| `dividir` | Equivalente a `str.split`, pero acepta expresiones regulares |
| `rsplit` | Equivalente a `str.rsplit`, pero acepta expresiones regulares |

Con estos podemos realizar una amplia gama de operaciones.
Por ejemplo, podemos extraer el nombre de cada elemento solicitando un grupo contiguo de caracteres al principio de cada elemento:

In [10]:
monte.str.extract('([A-Za-z]+)', expand=False)

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

O podemos hacer algo más complicado, como encontrar todos los nombres que comienzan y terminan con una consonante, utilizando los caracteres de expresión regular de inicio de cadena (`^`) y fin de cadena (`$`):

In [11]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

La capacidad de aplicar de manera concisa expresiones regulares en las entradas de "Series" o "DataFrame" abre muchas posibilidades para el análisis y la limpieza de datos.

### Métodos varios
Finalmente, existen algunos métodos diversos que permiten otras operaciones convenientes:

| Método | Descripción |
|--------|-------------|
| `obtener` | Indexa cada elemento |
| `rebanada` | Corta cada elemento|
| `slice_replace` | Reemplaza el segmento en cada elemento con el valor pasado|
| `gato` | Concatena cadenas|
| `repetir` | Repite valores |
| `normalizar` | Devuelve la forma Unicode de cadenas |
| `almohadilla` | Agrega espacios en blanco a la izquierda, a la derecha o a ambos lados de las cadenas|
| `envolver` | Divide cadenas largas en líneas con una longitud menor que un ancho determinado |
| `unirse` | Une cadenas en cada elemento de la `Serie` con el separador pasado|
| `get_dummies` | Extrae variables ficticias como un `DataFrame` |

#### Acceso y corte de elementos vectorizados

Las operaciones `get` y `slice`, en particular, permiten el acceso a elementos vectorizados desde cada matriz.
Por ejemplo, podemos obtener una porción de los primeros tres caracteres de cada matriz usando `str.slice(0, 3)`.
Tenga en cuenta que este comportamiento también está disponible a través de la sintaxis de indexación normal de Python; por ejemplo, `df.str.slice(0, 3)` es equivalente a `df.str[0:3]`:

In [12]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

La indexación mediante `df.str.get(i)` y `df.str[i]` también son similares.

Estos métodos de indexación también le permiten acceder a elementos de matrices devueltas por "split".
Por ejemplo, para extraer el apellido de cada entrada, podemos combinar la indexación `split` con `str`:

In [13]:
monte.str.split().str[-1]

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

#### Variables indicadoras

Otro método que requiere un poco de explicación adicional es el método `get_dummies`.
Esto es útil cuando sus datos tienen una columna que contiene algún tipo de indicador codificado.
Por ejemplo, podríamos tener un conjunto de datos que contenga información en forma de códigos, como A = "nacido en Estados Unidos", B = "nacido en el Reino Unido", C = "le gusta el queso", D = "le gusta el spam". :

In [14]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C',
                                    'B|D', 'B|C', 'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


La rutina `get_dummies` nos permite dividir estas variables indicadoras en un `DataFrame`:

In [15]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


Con estas operaciones como componentes básicos, puede construir una gama infinita de procedimientos de procesamiento de cadenas al limpiar sus datos.

No profundizaremos más en estos métodos aquí, pero le recomiendo que lea ["Trabajar con datos de texto"](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html) en la documentación en línea de Pandas, o consultar los recursos enumerados en [Recursos adicionales] (03.13-Further-Resources.ipynb).

## Ejemplo: base de datos de recetas

Estas operaciones de cadenas vectorizadas se vuelven más útiles en el proceso de limpieza de datos confusos del mundo real.
Aquí mostraré un ejemplo de eso, utilizando una base de datos de recetas abierta compilada de varias fuentes en la web.
Nuestro objetivo será analizar los datos de la receta en listas de ingredientes, para que podamos encontrar rápidamente una receta basada en algunos ingredientes que tenemos a mano. Los scripts utilizados para compilar esto se pueden encontrar en https://github.com/fictivekin/openrecipes, y el enlace a la versión más reciente de la base de datos también se encuentra allí.

Esta base de datos tiene unos 30 MB y se puede descargar y descomprimir con estos comandos:

In [16]:
# repo = "https://raw.githubusercontent.com/jakevdp/open-recipe-data/master"
# !cd data && curl -O {repo}/recipeitems.json.gz
# !gunzip data/recipeitems.json.gz

La base de datos está en formato JSON, por lo que usaremos `pd.read_json` para leerla (se requiere `lines=True` para este conjunto de datos porque cada línea del archivo es una entrada JSON):

In [17]:
recipes = pd.read_json('data/recipeitems.json', lines=True)
recipes.shape

(173278, 17)

Vemos que hay casi 175.000 recetas y 17 columnas.
Echemos un vistazo a una fila para ver qué tenemos:

In [18]:
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

Hay mucha información allí, pero gran parte de ella está en un formato muy desordenado, como es típico de los datos extraídos de la web.
En particular, la lista de ingredientes está en formato de cadena; Tendremos que extraer cuidadosamente la información que nos interesa.
Empecemos por echar un vistazo más de cerca a los ingredientes:

In [19]:
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

¡Las listas de ingredientes tienen una longitud promedio de 250 caracteres, con un mínimo de 0 y un máximo de casi 10,000 caracteres!

Sólo por curiosidad, veamos qué receta tiene la lista de ingredientes más larga:

In [20]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

Podemos hacer otras exploraciones agregadas; por ejemplo, podemos ver cuántas de las recetas son para alimentos para el desayuno (usando la sintaxis de expresiones regulares para hacer coincidir letras minúsculas y mayúsculas):

In [21]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

O ¿cuántas de las recetas incluyen la canela como ingrediente?

In [22]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

Incluso podríamos ver si alguna receta escribe mal el ingrediente como "cinamon":

In [23]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

Este es el tipo de exploración de datos que es posible con las herramientas de cadena de Pandas.
Es en esta manipulación de datos en lo que Python realmente sobresale.

### Un recomendador de recetas simple

Vayamos un poco más allá y comencemos a trabajar en un sistema simple de recomendación de recetas: dada una lista de ingredientes, queremos encontrar recetas que utilicen todos esos ingredientes.
Si bien es conceptualmente sencilla, la tarea se complica por la heterogeneidad de los datos: no existe una operación sencilla, por ejemplo, para extraer una lista limpia de ingredientes de cada fila.
Entonces, haremos un poco de trampa: comenzaremos con una lista de ingredientes comunes y simplemente buscaremos si están en la lista de ingredientes de cada receta.
Para simplificar, por el momento nos limitaremos a las hierbas y especias:

In [24]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

Luego podemos construir un `DataFrame` booleano que consta de valores `Verdadero` y `Falso`, indicando si cada ingrediente aparece en la lista:

In [25]:
import re
spice_df = pd.DataFrame({
    spice: recipes.ingredients.str.contains(spice, re.IGNORECASE)
    for spice in spice_list})
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


Ahora, como ejemplo, digamos que nos gustaría encontrar una receta que use perejil, pimentón y estragón.
Podemos calcular esto muy rápidamente usando el método ``query` de ``DataFrame``s, que se analiza más adelante en [Pandas de alto rendimiento: `eval()` y `query()`](03.12-Performance-Eval-and- Consulta.ipynb):

In [26]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

Encontramos sólo 10 recetas con esta combinación. Usemos el índice devuelto por esta selección para descubrir los nombres de esas recetas:

In [27]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object

Ahora que hemos reducido nuestra selección de recetas de 175.000 a 10, estamos en condiciones de tomar una decisión más informada sobre lo que nos gustaría cocinar para la cena.

### Yendo más allá con recetas

Esperamos que este ejemplo le haya dado una idea (je) de los tipos de operaciones de limpieza de datos que se habilitan de manera eficiente mediante los métodos de cadena de Pandas.
¡Por supuesto, construir un sistema sólido de recomendación de recetas requeriría *mucho* más trabajo!
Extraer listas completas de ingredientes de cada receta sería una parte importante de la tarea; Desafortunadamente, la amplia variedad de formatos utilizados hace que este sea un proceso que requiere relativamente mucho tiempo.
Esto apunta a la perogrullada de que en la ciencia de datos, la limpieza y manipulación de datos del mundo real a menudo comprende la mayor parte del trabajo, y Pandas proporciona las herramientas que pueden ayudarle a hacerlo de manera eficiente.